<h1> <center> Table of Contents </center> </h1>

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#1.-Importing-Required-Libraries-and-Data"> Importing Required Libraries and Data</a></li>
        <li><a href="#2.-Data-Pre-processing-and-Preparation"> Data Pre-processing and Preparation</a></li>
        <li><a href="#3.-Different-Methods">Different Methods</a> </li>
        <li><a href="#4.-Summary-of-Different-Methods">Summary of Different Methods</a></li>                 
      </li>
    </ol>
</div>

# 1. Importing Required Libraries and Data

In [4]:
# pip installs
# pip install python-Levenshtein
# pip install "fuzzywuzzy==0.18.0"
# pip install rapidfuzz
# pip install spacy
# pip install gensim
# !pip install python-Levenshtein
# from rank_bm25 import BM25Okapi
# nltk.download('punkt')
# nltk.download('stopwords') 
# nltk.download('wordnet')
# !pip install rank_bm25
# !python -m spacy download en_core_web_lg 
# !pip install fast-autocomplete

from gensim.utils import simple_preprocess
import spacy
from tqdm import tqdm

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import rapidfuzz
from rapidfuzz import process, utils

import re
from re import sub

from nltk.tokenize import word_tokenize
import nltk

from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

import spacy
from rank_bm25 import BM25Okapi
from tqdm import tqdm

import pandas as pd
from fast_autocomplete import AutoComplete

from pandas.core.common import flatten

In [5]:
#importing the file

df = pd.read_csv('microstrategy_and_dashboard.csv')
df_metric_desc = pd.read_csv('tableau_metric.csv')

In [6]:
# converting column values to lowercase

df = df.apply(lambda x: x.astype(str).str.lower())
df_metric_desc = df_metric_desc.apply(lambda x: x.astype(str).str.lower())

In [7]:
#converting global usage to numeric

df.Global_usage = pd.to_numeric(df.Global_usage)

In [8]:
# splitting caption and descriptors columns 

df = df[['dashboards','caption','Global_usage']]
df['metric_names'] = df.caption.str.split('|')
df_metric_desc['descriptor'] = df_metric_desc.descriptors.str.split('|')


In [9]:
# converting metric names and descriptors in each row values to multiple rows

df_split_1 = df.set_index(['dashboards'])['metric_names'].apply(pd.Series).stack().reset_index().drop('level_1', axis=1).rename(columns={0:'caption'})
df_split_2 = df_metric_desc.set_index(['metric_name'])['descriptor'].apply(pd.Series).stack().reset_index().drop('level_1', axis=1).rename(columns={0:'descriptor'})


In [10]:
# merging metrics and descriptors

df_final = df_split_1.merge(df_split_2, how = 'inner', left_on ='caption', right_on = 'metric_name')[['dashboards','caption','descriptor']]
df_final


,dashboards,caption,descriptor
0,ad analysis,beer style level value,macro style value
1,ad analysis,beer style level value,mezzo style value
2,ad analysis,beer style level value,micro style valu
3,ad analysis,beer style level value,beer style value
4,ad analysis,beer style level value,altbier
...,...,...,...
236553,surveyed stores,surveyed stores,2018
236554,surveyed stores,surveyed stores,2019
236555,surveyed stores,surveyed stores,2020
236556,surveyed stores,surveyed stores,2021


In [11]:
# checking for metric and descriptors

df_final[(df_final['dashboards'] == 'volume rank comp') | (df_final['descriptor'] == 'volume rank comp') | (df_final['caption'] == 'volume rank comp')]

,dashboards,caption,descriptor
3060,volume rank comp,beer style level value,macro style value
3061,volume rank comp,beer style level value,mezzo style value
3062,volume rank comp,beer style level value,micro style valu
3063,volume rank comp,beer style level value,beer style value
3064,volume rank comp,beer style level value,altbier
...,...,...,...
23741,volume rank comp,price per case,79
23742,volume rank comp,price per case,80
23781,volume rank comp,geography,geography description
30029,volume rank comp,brand family colors,vendor


### Creating Corpus

In [12]:
# creating corpus

dashboard_names_list = list(df_final.dashboards.unique())
metric_names_list = list(df_final.caption.unique())
descriptors_list = list(df_final.descriptor.unique())
text = dashboard_names_list + metric_names_list + descriptors_list

In [13]:
# pre-processing corpus

text = list(filter(None,text))
text

['ad analysis',
 'ad recap',
 'aggregate sales per pt vs. cwd',
 'brands on ad',
 'brewery comparisons',
 'company ',
 'company brands over time',
 'r geography',
 'r geography over time',
 'company',
 'category and segment analysis',
 'competitive set',
 'dimensions over time',
 'dollar rank comp',
 'dollar ranker with cwd',
 'dollar vs. ad share',
 'dollars vs. growth',
 'feature ad count',
 'footprint',
 'line geogs over time',
 'merch efficiency',
 'merch efficiency c&amp;r',
 'merch execution',
 'merch ranker',
 'merch vs. non-merch',
 'our brands',
 'our brands dollar ranker',
 'package analysis',
 'price analysis',
 'price bucket analysis',
 'quick scorecard',
 'rankers',
 'sales per pt vs. cwd',
 'style analysis',
 'topline',
 'velocity',
 'volume rank comp',
 'market index analysis',
 'market index analysis visual',
 'price promotion count',
 'the humbler',
 'brand control',
 'item audit',
 'market share',
 'our packages',
 'depletions month cube with inventory and shipmentmen

# 2. Data Pre-processing and Preparation

- stop words
- lower case
- stemming
- lemmatization
- noun filter 

## Search suggestions flow :
1. Suggestion (includes dashboard title, metric names and descriptors)
- Extract dashboard names list  
- Final suggestions 

## Efficient approaches:

- storing metric and descriptors as keys, and dashboards names as values in a dictionary 
- weightage to dashboard names, followed by metric and descriptors

##  Search Engine using :

- Regex expressions
- Fuzzyuwuzzy string matching
- Global Usage
- Auto-Complete
- Wordnet


# 3. Different Methods

## 3.1 Method 1: RegEx Expressions

In [14]:
# regex string matching

def regex_suggestions(input_query, corpus):
    return [x for x in corpus if re.search(input_query.lower(),x)]

In [15]:
def dashboard_names_suggestion(df, suggestions):
   
    dashboard_names = []
       
    for i in suggestions:
       
        if i in dashboard_names_list:
            
            dashboard_names.append(i)
        
        elif i in metric_names_list:
            
            dashboard_names = dashboard_names + (df.loc[(df.caption.str.lower() == i)].dashboards).to_list()
        
        else:
            
            dashboard_names = dashboard_names + (df.loc[(df.descriptor.str.lower() == i)].dashboards).to_list()

    return dashboard_names


In [16]:
# regex input query

inputQuery = 'volume'
regexSuggestionsList = regex_suggestions(inputQuery, text)
print(regexSuggestionsList)

# regex search output

regexDashboardsList = list(dashboard_names_suggestion(df_final, regexSuggestionsList))

regexSearchOutput = []
regexSearchOutput = list(dict.fromkeys(regexDashboardsList))
regexSearchOutput[:10]

['volume rank comp', 'rank (volume)', 'volume sales', 'volume sales change ya', 'volume sales per point', 'volume sales per percent last year', 'volume sales per pt change ya', 'volume sale']


['volume rank comp',
 'quick scorecard',
 'ad analysis',
 'r geography over time',
 'category and segment analysis',
 'competitive set',
 'dimensions over time',
 'line geogs over time',
 'market share',
 'package analysis']

## 3.2 Method 2: fuzzywuzzy String Matching

In [14]:
def fuzzywuzzy_suggestion(corpus, input_query):
   
    for i in [corpus]:
        
        return process.extract(input_query, i, limit=100)

In [15]:
# Input query:

InputQuery = 'Price per unit '

fuzzywuzzySuggestions = fuzzywuzzy_suggestion(text,InputQuery)

fuzzywuzzySuggestionsList = []
for i in fuzzywuzzySuggestions:
    fuzzywuzzySuggestionsList.append(i[0])


fuzzywuzzyDashboardsList = list(dashboard_names_suggestion(df_final, fuzzywuzzySuggestionsList))

fuzzywuzzySearchOutput = []
fuzzywuzzySearchOutput = list(dict.fromkeys(fuzzywuzzyDashboardsList))
fuzzywuzzySearchOutput[:10]

['dollar rank comp',
 'dollar ranker with cwd',
 'price analysis',
 'price bucket analysis',
 'rankers',
 'velocity',
 'depletions month cube with inventory and shipmentments',
 'depletionsletions cube',
 'retail month cube',
 'retail cube to  brand quality level']

In [16]:
# start_time = datetime.now()
# suggestion = fuzzywuzzy_suggestion(text,'rever')
# print(suggestion)

# dashboard_suggestions = list(dashboard_names_suggestion(df_final,suggestion))

# search_output = []
# search_output = list(dict.fromkeys(dashboard_suggestions))
# search_output
# end_time = datetime.now()
# print('Duration: {}'.format(end_time - start_time))

## 3.3 Method 3: Global Usage

In [17]:
def popular_dashboard_suggestion(df, globalUsageDashboardList): 
    
    return df[df['dashboards'].isin(globalUsageDashboardList)].sort_values(by = 'Global_usage',ascending = False)['dashboards'].tolist()            

In [18]:
# popular dashboard suggestions
 
globalUsageSeachOutput = popular_dashboard_suggestion(df,fuzzywuzzySearchOutput)
globalUsageSeachOutput[:10]

['dollar ranker with cwd',
 'category and segment analysis',
 'dollar rank comp',
 'style analysis',
 'rankers',
 'price analysis',
 'market index analysis',
 'ad analysis',
 'brewery comparisons',
 'distribution gains and losses cube vs. last month']

In [19]:
# checking for metric and descriptors

df_dashboard_suggestions = df_final[(df_final['dashboards'] == '') | (df_final['descriptor'] == '214 west main st') | (df_final['caption'] == '')]
df_dashboard_suggestions.dashboards.nunique()

15

## 3.4 Method 4: Fast-AutoComplete

In [20]:
# corpus dictionary creations

dashboard_dict = dict(enumerate(df_final.dashboards.unique().flatten(), 1))
dashboard_dict = {v: k  for k,v in dashboard_dict.items()}
dashboard_dict

metric_dict = dict(enumerate(df_final.caption.unique().flatten(), 1))
metric_dict = {v: k  for k,v in metric_dict.items()}
metric_dict

descriptor_dict = dict(enumerate(df_final.descriptor.unique().flatten(), 1))
descriptor_dict = {v: k  for k,v in descriptor_dict.items()}
descriptor_dict

masterDictionary = {}
masterDictionary.update(dashboard_dict)
masterDictionary.update(metric_dict)
masterDictionary.update(descriptor_dict)


for k,v in masterDictionary.items():
    masterDictionary[k] = {}
    

autocomplete = AutoComplete(words = masterDictionary)

# metric_dict = df_final.caption.unique().to_dict()
# descriptor_dict = df_final.descriptor.unique().to_dict()

In [21]:
# input to autocomplete

input_query = 'volumne'

autoCompleteSuggetionsList = autocomplete.search(word = input_query, max_cost= 3, size = 100)
autoCompleteSuggetionsList = list(flatten(autoCompleteSuggetionsList))

# autossuggestions

autoCompleteDashboardsList = dashboard_names_suggestion(df_final,autoCompleteSuggetionsList)

autoCompleteSearchOutput = []
autoCompleteSearchOutput = list(dict.fromkeys(autoCompleteDashboardsList))
autoCompleteSearchOutput

['ad analysis',
 'aggregate sales per pt vs. cwd',
 'brands on ad',
 'brewery comparisons',
 'company ',
 'company brands over time',
 'r geography',
 'r geography over time',
 'company',
 'category and segment analysis',
 'competitive set',
 'dimensions over time',
 'dollar rank comp',
 'dollar ranker with cwd',
 'dollar vs. ad share',
 'dollars vs. growth',
 'feature ad count',
 'line geogs over time',
 'merch vs. non-merch',
 'our brands',
 'our brands dollar ranker',
 'package analysis',
 'price bucket analysis',
 'price promotion count',
 'rankers',
 'sales per pt vs. cwd',
 'style analysis',
 'topline',
 'velocity',
 'volume rank comp',
 'market share',
 'quick scorecard']

## 3.5 Method 5: Wordnet

In [20]:
# Wordnet

from nltk.corpus import wordnet as wn

def getSynonyms(word1):
        synonymList1 = []
        for data1 in word1:
              wordnetSynset1 = wn.synsets(data1)
              tempList1=[]
              for synset1 in wordnetSynset1:
                 for synWords1 in synset1.lemma_names():
                     tempList1.append(synWords1)
              synonymList1.append(tempList1)
        return (synonymList1)
    
    
getSynonyms(['sales'])

[['gross_sales',
  'gross_revenue',
  'sales',
  'sale',
  'sale',
  'sale',
  'cut-rate_sale',
  'sales_event',
  'sale',
  'sale',
  'sales_agreement']]

# 4. Summary of Different Methods

In [23]:
def finalSuggestion(input_query):
        
        #regex
        
        regexSuggestionsList = regex_suggestions(input_query, text)
        regexDashboardsList = list(dashboard_names_suggestion(df_final, regexSuggestionsList))
        regexSearchOutput = []
        regexSearchOutput = list(dict.fromkeys(regexDashboardsList))
        
        #fuzzywuzzy
        
        fuzzywuzzySuggestions = fuzzywuzzy_suggestion(text,input_query)
        fuzzywuzzySuggestionsList = []
        for i in fuzzywuzzySuggestions:
            fuzzywuzzySuggestionsList.append(i[0])
        fuzzywuzzyDashboardsList = list(dashboard_names_suggestion(df_final, fuzzywuzzySuggestionsList))
        fuzzywuzzySearchOutput = []
        fuzzywuzzySearchOutput = list(dict.fromkeys(fuzzywuzzyDashboardsList))
        
        #using Global Usage

        globalUsageSearchOutput = popular_dashboard_suggestion(df,fuzzywuzzySearchOutput)

        #autossuggestions
        
        autoCompleteSuggetionsList = autocomplete.search(word = input_query, max_cost= 3, size = 100)
        autoCompleteSuggetionsList = list(flatten(autoCompleteSuggetionsList))
        autoCompleteDashboardsList = dashboard_names_suggestion(df_final,autoCompleteSuggetionsList) 
        autoCompleteSearchOutput = []
        autoCompleteSearchOutput = list(dict.fromkeys(autoCompleteDashboardsList))
        
        return regexSearchOutput[:10],fuzzywuzzySearchOutput[:10],autoCompleteSearchOutput[:10],globalUsageSearchOutput[:10]

In [24]:
searchOutputs = finalSuggestion('volume')

column_names = ['regex', 'fuzzywuzzy', 'fuzzywuzzy_with_autocomplete','fuzzywuzzy_with_globalusage']
finalSuggestionDF = pd.DataFrame(searchOutputs).transpose().set_axis(column_names, axis =1)
finalSuggestionDF

,regex,fuzzywuzzy,fuzzywuzzy_with_autocomplete,fuzzywuzzy_with_globalusage
0,volume rank comp,volume rank comp,ad analysis,dollar ranker with cwd
1,quick scorecard,quick scorecard,aggregate sales per pt vs. cwd,category and segment analysis
2,ad analysis,ad analysis,brands on ad,dollar rank comp
3,r geography over time,r geography over time,brewery comparisons,style analysis
4,category and segment analysis,category and segment analysis,company,rankers
5,competitive set,competitive set,company brands over time,price analysis
6,dimensions over time,dimensions over time,r geography,market index analysis
7,line geogs over time,line geogs over time,r geography over time,ad analysis
8,market share,market share,company,brewery comparisons
9,package analysis,package analysis,category and segment analysis,distribution gains and losses cube vs. last month


In [ ]:
# def final_suggestion(input_query, version):
   
#     if version == 'FuzzyWuzzy':
        
#         suggestion = fuzzywuzzy_suggestion(text,'3361')
#         dashboard_suggestions = list(dashboard_names_suggestion(df_final,suggestion))

#         search_output = []
#         search_output = list(dict.fromkeys(dashboard_suggestions))
#         return search_output
    
#     else version == 'regex'
#         return regex_suggestions('sales', text)
    
#     else version == 'fuzzyWith'


## Versions:

    1.Regex expression
    2.Fuzzywuzzy
    3.Fuzzywuzzy + Global_Usage
    5.AutoComplete + FuzzyWuzzy
    6.Wordnet + FuzzyWuzzy
    7.AutoComplete + Wordnet + FuzzyWuzzy + Global_Usage
    8.ElasticSearch 
    9.Doc2vec

## Search query conditions ??

- As the user is more concerned about the dashboard title, the search query should first retrieve the most appropriate dashboard name

- If user searches with descriptors or metric then the corresponding dashboard names should be suggested along with metric

- Dictionary with dashboard names as keys,metric_names,descriptors as values & displaying corresponding keys for value in search suggestion

- output string: dashboard name, metric

# Using TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

search_terms = 'dollar sales'
documents = text

doc_vectors = TfidfVectorizer().fit_transform([search_terms] + documents)

cosine_similarities = linear_kernel(doc_vectors[0:1], doc_vectors).flatten()
document_scores = [item.item() for item in cosine_similarities[1:]]

tuples = list(zip(text,document_scores))
df_scores = pd.DataFrame(tuples, columns = ['text','document_scores'])
df_scores.sort_values('document_scores', ascending =False).head(10)

In [ ]:
doc_vectors.nonzero()

# Using Glove's Model

In [ ]:
from re import sub
from gensim.utils import simple_preprocess

query_string = 'area'
documents = text

stopwords = ['the', 'and', 'are', 'a']

# From: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/soft_cosine_tutorial.ipynb
def preprocess(doc):
    # Tokenize, clean up input document string
    doc = sub(r'<img[^<>]+(>|$)', " image_token ", doc)
    doc = sub(r'<[^<>]+(>|$)', " ", doc)
    doc = sub(r'\[img_assist[^]]*?\]', " ", doc)
    doc = sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', " url_token ", doc)
    return [token for token in simple_preprocess(doc, min_len=0, max_len=float("inf")) if token not in stopwords]

# Preprocess the documents, including the query string
corpus = [preprocess(document) for document in documents]
query = preprocess(query_string)




# Build the term dictionary, TF-idf model
dictionary = Dictionary(corpus+[query])
tfidf = TfidfModel(dictionary=dictionary)

# Create the term similarity matrix.  
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)

# Compute Soft Cosine Measure between the query and the documents.
# From: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/soft_cosine_tutorial.ipynb
query_tf = tfidf[dictionary.doc2bow(query)]

index = SoftCosineSimilarity(
            tfidf[[dictionary.doc2bow(document) for document in corpus]],
            similarity_matrix)

doc_similarity_scores = index[query_tf]

# Output the sorted similarity scores and documents
sorted_indexes = np.argsort(doc_similarity_scores)[::-1]
for idx in sorted_indexes:
    print(f'{idx} \t {doc_similarity_scores[idx]:0.3f} \t {documents[idx]}')


In [ ]:
# import gensim.downloader as api
# from gensim.corpora import Dictionary
# from gensim.models import TfidfModel
# from gensim.models import WordEmbeddingSimilarityIndex
# from gensim.similarities import SparseTermSimilarityMatrix
# from gensim.similarities import SoftCosineSimilarity

# # Load the model: this is a big file, can take a while to download and open
# glove = api.load("glove-wiki-gigaword-50")    
# similarity_index = WordEmbeddingSimilarityIndex(glove)



In [ ]:
# Build the term dictionary, TF-idf model
dictionary = Dictionary(corpus+[query])
tfidf = TfidfModel(dictionary=dictionary)

# Create the term similarity matrix.  
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)

# Compute Soft Cosine Measure between the query and the documents.
# From: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/soft_cosine_tutorial.ipynb
query_tf = tfidf[dictionary.doc2bow(query)]

index = SoftCosineSimilarity(
            tfidf[[dictionary.doc2bow(document) for document in corpus]],
            similarity_matrix)

doc_similarity_scores = index[query_tf]

# Output the sorted similarity scores and documents
sorted_indexes = np.argsort(doc_similarity_scores)[::-1]
for idx in sorted_indexes:
    print(f'{idx} \t {doc_similarity_scores[idx]:0.3f} \t {documents[idx]}')


# Docsim

## Other Word Embedding techniques:
BERT, BM25,fasttext, FAISS, SEQ2SEQ, WORD2VEC,NMSLIB

In [ ]:
from IPython.display import Image
Image(filename=r'D:\internship Experience\Fintech 2021\Different_techniques.png', width = 500, height =100)